In [ ]:
!pip install openpyxl

In [ ]:
!pip install --upgrade jupyter ipywidgets

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
xlsx_file ='for_Train_and_validation_data.xlsx' 
df = pd.read_excel(xlsx_file)
df.head()

In [ ]:
df.is_toxic.value_counts()

In [ ]:
possible_labels = df.is_toxic.unique()
possible_labels

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

In [ ]:
df['label'] = df.is_toxic.replace(label_dict)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [ ]:
X_train

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df

In [ ]:
df.groupby(['is_toxic', 'label', 'data_type']).count()

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
df[df.data_type=='train'].message.values

In [ ]:
df.iloc[X_val[0]]

In [ ]:
df.iloc[y_val[0]]

In [ ]:
X_val[0]

In [ ]:
# from transformers import AutoTokenizer
# import torch
# import pandas as pd

# # Replace 'model_name' with the name of the pre-trained model you're using
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Assuming you have a DataFrame named 'df'
# for index, row in df.iterrows():
#     data_type = row['data_type']
#     message = row['message']
#     label = row['label']

#     encoded_data = tokenizer.encode_plus(
#         text=message,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=256,
#         return_tensors='pt'
#     )

#     input_ids = encoded_data['input_ids']
#     attention_mask = encoded_data['attention_mask']
    
#     #print(f"Processing row {index + 1} - Data Type: {data_type}, Message: {message}, Label: {label}")
    
#     # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'
 

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=1024, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=1024, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

In [ ]:
len(dataset_val)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 8

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
val_loss, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('ToxiCR_BERT.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
len(predictions), len(true_vals)

In [ ]:
from sklearn import metrics

In [ ]:
print("Accuracy:",metrics.accuracy_score(preds_flat, true_vals))
print("Precision:",metrics.precision_score(preds_flat, true_vals))
print("Recall:",metrics.recall_score(preds_flat, true_vals))
print("f1 score", metrics.f1_score(preds_flat, true_vals, average='weighted'))

### Testing

In [ ]:
xlsx_file ='prediction_github_projectsComment/aggregated_data.csv' 
#df_test = pd.read_excel(xlsx_file)
df_test = pd.read_csv(xlsx_file)
df_test.head()

In [ ]:
possible_labels = df_test.is_toxic.unique()
possible_labels

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df_test['label'] = df_test.is_toxic.replace(label_dict)

In [ ]:
encoded_test_val = tokenizer.batch_encode_plus(
    df_test.message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

In [ ]:
input_ids_test = encoded_test_val['input_ids']
attention_masks_test = encoded_test_val['attention_mask']
labels_test = torch.tensor(df_test.label.values)

In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
batch_size = 32
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
def test_model(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }  
        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
pred_test = test_model(dataloader_test) 
preds_flat_test = np.argmax(pred_test, axis=1).flatten()
print(preds_flat_test)

In [ ]:
import pandas as pd
pred_data = pd.DataFrame(preds_flat_test, columns = ["prediction"])
pd_cont = pd.concat([df_test["message"], pred_data], axis = 1)
pd_cont.to_csv("prediction_github_comments_byToxicR.csv")

In [ ]:
len(pred_data)

In [ ]:
len(X_val)

In [ ]:
pred_data

In [ ]:
df_slice = df_test["message"]
df_slice.reset_index()

In [ ]:
pd_cont_rst = pd.concat([df_slice.reset_index(), pred_data], axis = 1)
pd_cont_rst.to_csv("done.csv")